# Nitrobacter Growth Curve Plotting for pH and temperature optima

Zach Flinkstrom - Jun2023

In [1]:
import pandas as pd
import numpy as np
import altair as alt

## Helper functions

In [2]:
def df_growth_rate(df, organism, time_start, time_end, column):
    '''Function to calculate growth rate from given dataframe with specified start and end times.
    Uses the slope of the natural log vs time to estimate.'''
    tmp = df[(df['Organism']==organism)&(df['Time_elapsed_hr']>time_start)&(df['Time_elapsed_hr']<time_end)]
    return np.polyfit(tmp['Time_elapsed_hr'], np.log(tmp[column]), 1)[0]

In [3]:
def df_yield(df, organism, time_start, time_end):
    '''Calculates growth yield in cells/pmol-N oxidized between given times'''
    tmp = df[(df['Organism']==organism)&(df['Time_elapsed_hr']>time_start)&(df['Time_elapsed_hr']<time_end)].sort_values(by='Time_elapsed_hr')
    cell_count_i = tmp.iloc[0,:]['Cell_count_cells-per-mL']
    cell_count_f = tmp.iloc[-1,:]['Cell_count_cells-per-mL']
    no2_i = tmp.iloc[0,:]['Nitrite_uM']
    no2_f = tmp.iloc[-1,:]['Nitrite_uM']
    return (cell_count_f - cell_count_i)/((no2_i - no2_f)/1e3*1e6)

# pH Screen

In [4]:
rep1 = pd.read_excel('data/25C_pH-screen-MLSDS22_1mM_nitrite.xlsx',sheet_name=0)
rep2 = pd.read_excel('data/25C_pH-screen-MLSDS22_1mM_nitrite.xlsx',sheet_name=1)
rep3 = pd.read_excel('data/25C_pH-screen-MLSDS22_1mM_nitrite.xlsx',sheet_name=2)
pH_data = pd.concat([rep1, rep2, rep3])
pH_data.head()

,Date,Organism,Replicate,pH,Start_date,Time_elapsed_hr,NH4_ug-N/L,Nitrite_ug-N/L,TON_ug-N/L,NH4_uM,Nitrite_uM,TON_uM,Nitrate_uM,Cell_count_total_volume,Cell_count_sample_volume,5000_events_uL_1,5000_events_uL_2,5000_events_uL_3,Cell_count_cells-per-mL
0,2023-10-13 15:00:00,MLSD-S22,1,6.0,2023-10-13 15:00:00,0.0,NaN,13648.21,13721,NaN,974.384950,979.581638,5.196687,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-10-13 15:00:00,MLSD-S22,2,6.0,2023-10-13 15:00:00,0.0,NaN,13653.81,13925,NaN,974.784750,994.145784,19.361034,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-10-13 15:00:00,MLSD-S22,3,6.0,2023-10-13 15:00:00,0.0,NaN,13738.06,13771,NaN,980.799600,983.151282,2.351681,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-10-13 15:00:00,MLSD-S22,1,6.5,2023-10-13 15:00:00,0.0,NaN,13561.71,13803,NaN,968.209467,985.435854,17.226387,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-10-13 15:00:00,MLSD-S22,2,6.5,2023-10-13 15:00:00,0.0,NaN,13586.68,13724,NaN,969.992147,979.795816,9.803670,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
pH_data['Nitrite_conc_mM'] = pH_data.Nitrite_uM/1000
pH_data['Nitrate_conc_mM'] = pH_data.Nitrate_uM/1000

In [6]:
no2 = alt.Chart(pH_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrite_conc_mM:Q', title='Nitrite (mM)')
            ).properties(height=150, width=100)

no2_layer = alt.layer(
    no2.transform_filter(alt.datum.Replicate == 1),
    no2.transform_filter(alt.datum.Replicate == 2),
    no2.transform_filter(alt.datum.Replicate == 3),
    no2.transform_filter(alt.datum.Replicate == 4),
    no2.transform_filter(alt.datum.Replicate == 5),
    no2.transform_filter(alt.datum.Replicate == 6)
).facet('pH:Q', title="MLSD-S22: pH",).resolve_scale(y='shared', x='shared').properties(spacing={'column':10})

no3 = alt.Chart(pH_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Nitrate_conc_mM:Q', title='Nitrate (mM)')
            ).properties(height=150, width=100)

no3_layer = alt.layer(
    no3.transform_filter(alt.datum.Replicate == 1),
    no3.transform_filter(alt.datum.Replicate == 2),
    no3.transform_filter(alt.datum.Replicate == 3),
    no3.transform_filter(alt.datum.Replicate == 4),
    no3.transform_filter(alt.datum.Replicate == 5),
    no3.transform_filter(alt.datum.Replicate == 6)
).facet('pH:Q').resolve_scale(y='shared', x='shared').properties(spacing={'column':10})

cell = alt.Chart(pH_data).mark_line(point=True, opacity=0.3, clip=True).encode(
            alt.X('Time_elapsed_hr:Q', title='Time (hr)'),
            alt.Y('Cell_count_cells-per-mL:Q', title='Cell Concentration (cells/mL)', axis=alt.Axis(format="2.0e"))
            ).properties(height=150, width=100)

cell_layer = alt.layer(
    cell.transform_filter(alt.datum.Replicate == 1),
    cell.transform_filter(alt.datum.Replicate == 2),
    cell.transform_filter(alt.datum.Replicate == 3),
    cell.transform_filter(alt.datum.Replicate == 4),
    cell.transform_filter(alt.datum.Replicate == 5),
    cell.transform_filter(alt.datum.Replicate == 6)
).facet('pH:Q').resolve_scale(y='shared', x='shared').properties(spacing={'column':10})

figure = alt.vconcat(no2_layer, no3_layer, cell_layer, center=True).configure_axis(labelFontSize=14, titleFontSize=14).configure_title(fontSize=18)
figure.save('figures/MLSD-S22_pH_growth_curves_facet.svg')
figure

alt.VConcatChart(...)

In [7]:
pHs = np.sort(list(set(pH_data.pH)))
reps = np.sort(list(set(pH_data.Replicate)))
growth_rate_df = pd.DataFrame(columns=['pH', 'Replicate', 'Growth_rate', 'Method'])

for pH in pHs:
    for rep in reps:
        if (pH >= 8.5)|(pH == 5.0):
            growth_rate = df_growth_rate(pH_data[(pH_data.pH==pH) & (pH_data.Replicate==rep)], 'MLSD-S22', 30, 150, 'Cell_count_cells-per-mL')
            growth_rate_df = pd.concat([growth_rate_df.T, pd.Series({'pH':pH, 'Replicate':rep, 'Growth_rate':growth_rate, 'Method':'Cell-count-based'})], axis=1).T
            growth_rate = df_growth_rate(pH_data[(pH_data.pH==pH) & (pH_data.Replicate==rep)], 'MLSD-S22', 30, 150, 'Nitrate_uM')
            growth_rate_df = pd.concat([growth_rate_df.T, pd.Series({'pH':pH, 'Replicate':rep, 'Growth_rate':growth_rate, 'Method':'Nitrate-based'})], axis=1).T
        else:
            growth_rate = df_growth_rate(pH_data[(pH_data.pH==pH) & (pH_data.Replicate==rep)], 'MLSD-S22', 30, 100, 'Cell_count_cells-per-mL')
            growth_rate_df = pd.concat([growth_rate_df.T, pd.Series({'pH':pH, 'Replicate':rep, 'Growth_rate':growth_rate, 'Method':'Cell-count-based'})], axis=1).T
            growth_rate = df_growth_rate(pH_data[(pH_data.pH==pH) & (pH_data.Replicate==rep)], 'MLSD-S22', 30, 100, 'Nitrate_uM')
            growth_rate_df = pd.concat([growth_rate_df.T, pd.Series({'pH':pH, 'Replicate':rep, 'Growth_rate':growth_rate, 'Method':'Nitrate-based'})], axis=1).T

In [8]:
point = alt.Chart(data=growth_rate_df[growth_rate_df.Method=='Cell-count-based']).mark_line(point=True).encode(
    alt.X('pH:Q', scale=alt.Scale(domain=[5,9])),
    alt.Y('mean(Growth_rate):Q', scale=alt.Scale(domain=[0,0.06]), title='Specific Growth Rate (hr\u207B\u2071)'))

error_bar = alt.Chart(data=growth_rate_df[growth_rate_df.Method=='Cell-count-based']).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('pH:Q'),
    alt.Y('Growth_rate:Q', title='Specific Growth Rate (hr\u207B\u2071)'))

figure = (point + error_bar).properties(width=300, height=200, title='Growth Rate pH Dependence').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/pH_growth_rate.svg')
figure

alt.LayerChart(...)

In [9]:
pHs = np.sort(list(set(pH_data.pH)))
reps = np.sort(list(set(pH_data.Replicate)))
yield_df = pd.DataFrame(columns=['Strain','pH', 'Replicate', 'Yield_cells-per-pmol-N'])

for pH in pHs:
    reps = np.sort(list(set(pH_data[pH_data['pH']==pH].Replicate)))
    for rep in reps:
            yield_value = df_yield(pH_data[(pH_data['pH']==pH) & (pH_data.Replicate==rep)], 'MLSD-S22', 0, 200)
            yield_df = pd.concat([yield_df.T, pd.Series({'Strain':'MLSD-S22','pH':pH, 'Replicate':rep, 'Yield_cells-per-pmol-N':yield_value})], axis=1).T

In [10]:
point = alt.Chart(data=yield_df).mark_line(point=True).encode(
    alt.X('pH:Q', scale=alt.Scale(domain=[5,9])),
    alt.Y('mean(Yield_cells-per-pmol-N):Q', scale=alt.Scale(domain=[0,5]), title='Growth Yield (cells/pmol-N)'))

error_bar = alt.Chart(data=yield_df).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True, clip=True).encode(
    alt.X('pH:Q'),
    alt.Y('Yield_cells-per-pmol-N:Q', title='Growth Yield (cells/pmol-N)'))

figure = (point + error_bar).properties(width=300, height=200, title='Growth Yield pH Dependence').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/pH_yield.svg')
figure

alt.LayerChart(...)

In [11]:
q_df = growth_rate_df.merge(yield_df, on=['pH','Replicate'])
q_df['uptake_rate'] = q_df['Growth_rate']/q_df['Yield_cells-per-pmol-N']

In [12]:
point = alt.Chart(data=q_df[q_df.Method=='Cell-count-based']).mark_line(point=True).encode(
    alt.X('pH'),
    alt.Y('mean(uptake_rate):Q', title=['Substrate Specific Uptake Rate','(pmolN cell\u207B\u2071 hr\u207B\u2071)']))

error_bar = alt.Chart(data=q_df[q_df.Method=='Cell-count-based']).mark_errorbar(extent='stdev', color=alt.HexColor('#1f77b4'), ticks=True).encode(
    alt.X('pH'),
    alt.Y('uptake_rate:Q', title=['Substrate Specific Uptake Rate','(pmolN cell\u207B\u2071 hr\u207B\u2071)']))

figure = (point + error_bar).properties(width=300, height=200, title='Uptake Rate Dependence on pH').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/pH_uptake_rate.svg')
figure

alt.LayerChart(...)

# Temperature optimum data from Qin Lab

In [13]:
ou_data = pd.read_excel('data/OU_temp_data.xlsx')
ou_data

,Temperature_C,Growth_rate_hr-1,stdev
0,5,0.000000,0.000000
1,7,0.001547,0.000200
2,15,0.012907,0.000501
3,20,0.022542,0.009849
4,25,0.031149,0.002488
5,27,0.038637,0.008645
6,30,0.031694,0.001183
7,37,0.000000,0.000000


In [16]:
point = alt.Chart(data=ou_data).mark_line(point=True).encode(
    alt.X('Temperature_C:Q', scale=alt.Scale(domain=[0,40])),
    alt.Y('Growth_rate_hr-1:Q', scale=alt.Scale(domain=[0,0.06]), title='Specific Growth Rate (hr\u207B\u2071)'))

error_bar = alt.Chart(data=ou_data).mark_errorbar(color=alt.HexColor('#1f77b4'), ticks=True).encode(
    x=alt.X('Temperature_C:Q', title='Temperature (\u1D52C)'),
    y=alt.Y('Growth_rate_hr-1:Q', scale=alt.Scale(domain=[0,0.06]), title='Specific Growth Rate (hr\u207B\u2071)'),
    yError='stdev:Q')

figure = (point + error_bar).properties(width=300, height=200, title='Growth Rate Temperature Dependence').configure_axis(
    labelFontSize=14, titleFontSize=14).configure_title(fontSize=14).configure_legend(labelFontSize=14, titleFontSize=14)
figure.save('figures/temp_growth_rate.svg')
figure

alt.LayerChart(...)